# Diatonic fuckery

I want to do some wacky shit with diatonic chords. Like, what about pivoting to other scales?

Let's say I'm in C major. I have a triad...

! Wait! I don't actually have ways of deriving pitch sets from pitch classes for the most part. Erm...

# But first, voicing pitch class sets into pitch sets

Okay, I'm gonna have to work this out. I mean, I kind of did in [chord voicings](../ideas/chord_voicings.ipynb). 
But idk...

It's kind of a weird problem, isn't it? Like, if I have a chord progression in the form of a sequence of pitch class sets, how the hell do I derive a sequence of pitch sets from this?

I think what trips me up is that there are so many possible parameters. There are so many possible voicings!

I mean, take a C major triad, for example. On a grand piano, you have 88 pitches: 21 through 108. 

Within these pitches, how many pitch sets contain at least one of each pitch class in a C major triad?

I think you must count how many times each pitch class occurs in this range, like so:

In [ ]:
c = [0, 4, 7]
pc_counts = [0, 0, 0]

for i in range(21, 108 + 1):
    match i % 12:
        case 0:
            pc_counts[0] += 1
        case 4:
            pc_counts[1] += 1
        case 7:
            pc_counts[2] += 1

print(list(zip(c, pc_counts)))

[(0, 8), (4, 7), (7, 7)]


So, C occurs 8 times, E occurs 7 times, and G occurs 7 times.

I think, to get the set of all pitch sets you can derive in this range, you simply take the powerset of each set of pitch classes (so the powerset of the set of all C's, the powerset of the set of all E's, and the powerset of the set of all G's), ignoring the empty set of course, and then you take the cartesian product of these 3 powersets, and union the sets in the resulting tuples.

Ahh, nice and confusing sounding. Well, here's an example of such a tuple, using Note-Octave notation: `({C3, C5}, {E3}, {G2, G6})`. There would be many of these tuples, and you would union them, giving `{G2, C3, E3, C5, G6}`. See what I mean? I sure hope so!

Anyways, we're not actually gonna generate this, I'm just counting right now.

So, how WOULD you count this? Well, the cardinality of a cartesian product `A X B` is equal to `card(A) * card(B)`. So if A has 4 elements and B has 3, then the cartesian product has 12 elements.

The cardianlity of the powerset of A is equal to `2^(card(A))`. If we're ignoring the empty set, then we subtract 1.

So, if we want to count how many voicings of a pitch class set exist in a pitch range, here's our algorithm...

In [13]:
from harmonica.pitch._scales import PitchClassSet


def count_voicings_in_range(
    pcset: PitchClassSet, lower_bound: int, upper_bound: int
) -> int:
    """Counts how many pitch sets can be derived from a pitch class set in a range of pitches,
    assuming each pitch class appears at least once in each pitch set."""

    pc_counts = [0] * pcset.cardinality

    for p in range(lower_bound, upper_bound + 1):
        for i, pc in enumerate(pcset.pitch_classes):
            if p % pcset.modulus == pc:
                pc_counts[i] += 1

    count = 1

    for pc_count in pc_counts:
        count *= (2**pc_count) - 1

    return count


c_triad = PitchClassSet([0, 4, 7], 12)
count = count_voicings_in_range(c_triad, 21, 108)
print(count)

4112895


Jesus Christ! That's a lot of voicings! 4,112,895.

You know what? Let's actually generate this set. Let's write a generator function that yields these pitch sets.

In [49]:
from itertools import chain, product, islice
from more_itertools import powerset
from typing import Iterator

from harmonica.pitch._changes import PitchSetSeq
from harmonica.pitch._pitchset import PitchSet
from harmonica.utility._gm import GM
from harmonica.utility._mixed import Mixed


def list_voicings_in_range(
    pcset: PitchClassSet, lower_bound: int, upper_bound: int
) -> Iterator[PitchSet]:
    """Yields all pitch sets that can be derived from a pitch class set in a range of pitches,
    assuming each pitch class appears at least once in each pitch set."""

    # List out each occurrence of each pitch class within bounds
    in_bounds_pitches_for_each_pc = (
        (p for p in range(lower_bound, upper_bound + 1) if p % pcset.modulus == pc)
        for pc in pcset.pitch_classes
    )

    for pitches in product(
        *(
            islice(powerset(in_bound_pitches), 1, None)
            for in_bound_pitches in in_bounds_pitches_for_each_pc
        )
    ):
        yield PitchSet(sorted(chain(*pitches)))


restricted = filter(
    lambda pset: len(pset.pitches) == 3,
    list_voicings_in_range(PitchClassSet([0, 4, 7], 12), 36, 96),
)

PitchSetSeq(list(restricted)).preview(
    duration=Mixed("1/2"), program=GM.AcousticGuitarNylon
)

Ain't that neat? No, I didn't stick to the grand piano range. I feel like octaves 3 through 7 are probably the most useful overall for generating chords in.

But wow! I even used iterators, very lazy!

Let's see it for more complicated chords.

In [55]:
chord = PitchClassSet([0, 5, 7, 11], 12)

restricted = filter(
    lambda pset: len(pset.pitches) == chord.cardinality,
    list_voicings_in_range(chord, 48, 84),
)

PitchSetSeq(list(restricted)).preview(duration=Mixed("2"), program=GM.Pad1NewAge)